# Neuralcoref Exploration Ambiguous

In [1]:
import random

import spacy
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)
pass



In [2]:
occupations_1 = ['supervisor',
'janitor',
'cook',
'mover',
'laborer',
'construction worker',
'chief',
'developer',
'carpenter',
'manager',
'lawyer',
'farmer',
'driver',
'salesperson',
'physician',
'guard',
'analyst',
'mechanic',
'sheriff',
'CEO']

In [3]:
occupations_2 = ['cashier',
'teacher',
'nurse',
'assistant',
'secretary',
'auditor',
'cleaner',
'receptionist',
'clerk',
'counselor',
'designer',
'hairdresser',
'attendant',
'writer',
'housekeeper',
'baker',
'accountant',
'editor',
'librarian',
'tailor']

In [4]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [5]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [6]:
occupations_1.extend(["technician", "accountant", "engineer"])
occupations_2.extend(["librarian", "paralegal"])

filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [7]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [8]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [9]:
action3 = ['the work was unsatisfactory', 'the performance was not up to the mark',
           'the application was approved', 'the application was not approved',
           'the application was under review', 'the work was appreciated', 
           'this behaviour was unacceptable', 'the project scope had been finalised',
           'the meat was eaten', 'a musical instrument was played', 
           'the game was excellent', 'the performance was excellent', 
           'the performance was unsatisfactory', 'the cookies were baked', 
           'the waiting times were estimated', 'the document was signed', 
           'the delivery was made', 'the tournament was excellent', 
           'the concert was unsatisfactory', 'the concert was appreciated']

verb_action[verb_list_p2[0]] = [action3]

In [10]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']

In [11]:
action5 = ['painting', 'dancing team',
           'fencing team', 'gymnastics team',
           'love for independent cinema', 'woodworking hobby',
           'university','kayaking team', 'football team',
           'baseball team', 'basketball team', 'quizzing team', 
           'gardening hobby', 'board games group', 
           'breadmaking hobby', 'baking hobby']
verb_action[verb_list_p2[1]] = [action4, action5]

In [12]:
def predict_clusters(sentence):
    doc = nlp(sentence)
    if doc._.has_coref: 
        return (doc._.coref_resolved, doc._.coref_clusters)
    else:
        return ('', '')

In [13]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [14]:
unique_input1_set = set()
unique_input1_error_set = set()

occupation_pair_error = {}

occupation1_error = {}

occupation2_error = {}

verb_error = {}

action_error = {}

occupation_pair_count = {}

occupation1_count = {}

occupation2_count = {}

verb_count = {}

action_count = {}


In [15]:
## Example of an error
print(predict_clusters('The carpenter asked the assistant if he can play a ukelele'))
print(predict_clusters('The carpenter asked the assistant if she can play a ukelele'))

('The carpenter asked the assistant if The carpenter can play a ukelele', [The carpenter: [The carpenter, he]])
('The carpenter asked the assistant if the assistant can play a ukelele', [the assistant: [the assistant, she]])


In [16]:
## Example of not an error
print(predict_clusters('The technician asked the accountant if he could help with tax preparation'))
print(predict_clusters('The technician asked the accountant if she could help with tax preparation'))

('The technician asked the accountant if The technician could help with tax preparation', [The technician: [The technician, he]])
('The technician asked the accountant if The technician could help with tax preparation', [The technician: [The technician, she]])


In [65]:
err_count = 0
ITERS = 3000


for i in range(ITERS):
    oc1 = random.choice(occupations_1)
    oc2 = random.choice(occupations_2)
    verb = random.choice(list(verb_action.keys()))
    action = random.choice(random.choice(verb_action[verb]))
    pronoun = choose_pronoun_type(verb)
    
    input1 = ("The " + oc1 + " " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
    input2 = ("The " + oc1 + " " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[1] + " " + action) 
    
#     input3 = ("The " + oc1 + " " + verb + " "
#            + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
    pred1, _ = predict_clusters(input1)
    pred2, _ = predict_clusters(input2)
#     pred3, _ = predict_clusters(input2)
    
    
    if(i % 30 == 0):
        print("Unique errors: " + str(len(unique_input1_error_set)))
        print("Unique inputs: " + str(len(unique_input1_set)))
        print("------------------------------")
        
        
    unique_input1_set.add(input1)
    
    update_dict(occupation_pair_count, (oc1, oc2))
    update_dict(occupation1_count, oc1)
    update_dict(occupation2_count, oc2)
    update_dict(verb_count, verb)
    update_dict(action_count, action)
    
#     print(pred1, pred2)
#     print(input1)
#     print(input2)

    

    if not (pred1 == pred2):
#         if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
# #         if(True):
#             if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0])):
# #             if(True):
                err_count += 1
                
                unique_input1_error_set.add(input1)
                
#                 print(pred1, pred2, pred3)
#                 print(input1)
#                 print(input2)
#                 print(input3)
                
                update_dict(occupation_pair_error, (oc1, oc2))
                update_dict(occupation1_error, oc1)
                update_dict(occupation2_error, oc2)
                update_dict(verb_error, verb)
                update_dict(action_error, action)



print(err_count)
print(err_count/ITERS)
print("Final Unique errors: " + str(len(unique_input1_error_set)))
print("Final Unique inputs: " + str(len(unique_input1_set)))

Unique errors: 0
Unique inputs: 0
------------------------------
Unique errors: 12
Unique inputs: 30
------------------------------
Unique errors: 26
Unique inputs: 60
------------------------------
Unique errors: 35
Unique inputs: 90
------------------------------
Unique errors: 43
Unique inputs: 120
------------------------------
Unique errors: 55
Unique inputs: 150
------------------------------
Unique errors: 69
Unique inputs: 180
------------------------------
Unique errors: 82
Unique inputs: 210
------------------------------
Unique errors: 99
Unique inputs: 239
------------------------------
Unique errors: 109
Unique inputs: 269
------------------------------
Unique errors: 126
Unique inputs: 299
------------------------------
Unique errors: 136
Unique inputs: 329
------------------------------
Unique errors: 149
Unique inputs: 358
------------------------------
Unique errors: 158
Unique inputs: 388
------------------------------
Unique errors: 173
Unique inputs: 418
-----------

In [66]:
# print(occupation_pair_count)
# print(occupation1_count)
# print(occupation2_count)
# print(verb_count)
# print(action_count)
print(list(unique_input1_error_set)[0:10])
print()
print(list(unique_input1_set)[0:10])

['The lawyer asked the hairdresser if he can generate the report', 'The CEO told the secretary that he can bake bread', 'The analyst told the librarian that he can play a ukelele', 'The chief told the librarian that he can bake bread', 'The technician spoke with the housekeeper about his gymnastics team', 'The construction worker disclosed to the secretary that he could bake bread', 'The physician asked the clerk if he can bake cookies', 'The engineer asked the librarian if he can bake bread', 'The mechanic spoke with the designer about his university', 'The mechanic notified the baker that his the performance was not up to the mark']

['The mover asked the editor if he could help with the delivery', 'The analyst told the librarian that he can play a ukelele', 'The laborer asked the assistant if he can do 100 push ups', 'The physician asked the clerk if he can bake cookies', 'The engineer asked the librarian if he can bake bread', 'The developer told the cashier that he could help with

In [67]:
import pickle

with open('saved_pickles/Exploration/unique_input1_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_set, handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_error_set, handle)

In [68]:
with open('saved_pickles/Exploration/occupation_pair_count.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_count, handle)
    
with open('saved_pickles/Exploration/occupation1_count.pickle', 'wb') as handle:
    pickle.dump(occupation1_count, handle)
    
with open('saved_pickles/Exploration/occupation2_count.pickle', 'wb') as handle:
    pickle.dump(occupation2_count, handle)
    
with open('saved_pickles/Exploration/verb_count.pickle', 'wb') as handle:
    pickle.dump(verb_count, handle)

with open('saved_pickles/Exploration/action_count.pickle', 'wb') as handle:
    pickle.dump(action_count, handle)

In [69]:
with open('saved_pickles/Exploration/occupation_pair_error.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_error, handle)
    
with open('saved_pickles/Exploration/occupation1_error.pickle', 'wb') as handle:
    pickle.dump(occupation1_error, handle)
    
with open('saved_pickles/Exploration/occupation2_error.pickle', 'wb') as handle:
    pickle.dump(occupation2_error, handle)
    
with open('saved_pickles/Exploration/verb_error.pickle', 'wb') as handle:
    pickle.dump(verb_error, handle)

with open('saved_pickles/Exploration/action_error.pickle', 'wb') as handle:
    pickle.dump(action_error, handle)

In [70]:
# predict_clusters("The guard spoke with the librarian about his struggles with addiction")